In [5]:
import csv
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import box

In [6]:
#Load the labels from the two files

labels_file1 = '/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/clean_crowd_benchmark.csv'
labels_file2 = '/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/expert/expertconsensus_superclass.csv'

In [8]:
# Define the IOU threshold and read in the labels
iou_thresh = 0.5

# Define dictionaries to map class IDs to class names
class_map1 = {1: 'Crane', 2: 'Goose', 3: 'Duck'}
class_map2 = {1: 'Crane', 2: 'Goose', 3: 'Duck'}

# Read in the labels from file 1
labels1 = defaultdict(list)
with open(labels_file1) as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header
    for row in reader:
        label_id, filename, bbox, class_id = row
        bbox = [float(x) for x in bbox[1:-1].split(',')]
        labels1[filename].append((label_id, bbox, int(class_id)))

# Read in the labels from file 2
labels2 = defaultdict(list)
with open(labels_file2) as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header
    for row in reader:
        label_id, filename, bbox, class_id = row
        bbox = [float(x) for x in bbox[1:-1].split(',')]
        labels2[filename].append((label_id, bbox, int(class_id)))

In [35]:
# Compute the confusion matrix
conf_matrix = np.zeros((len(class_map1), len(class_map2)), dtype=int)
for filename in set(labels1.keys()) & set(labels2.keys()):
    if filename in labels1 and filename in labels2:
        for label1 in labels1[filename]:
            for label2 in labels2[filename]:
                bbox1 = box(*label1[1])
                bbox2 = box(*label2[1])
                iou = bbox1.intersection(bbox2).area / bbox1.union(bbox2).area
                if iou >= iou_thresh and label1[2] == label2[2]:
                    conf_matrix[label1[2] - 1, label2[2] - 1] += 1

In [36]:
# Print the confusion matrix
print('Confusion Matrix:')
print(conf_matrix)

Confusion Matrix:
[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [ ]:
# Visualize the confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', ax=ax)

# Add labels and title
ax.set_xlabel('Labels2')
ax.set_ylabel('Labels1')
ax.set_title('Confusion Matrix')

# Add class names as tick labels
ax.set_xticklabels(list(class_map2.values()))
ax.set_yticklabels(list(class_map1.values()))

plt.show()